# Import Your Own Data

## Import Packages

In [50]:
%reload_ext autoreload
from util.dependencies import *
from settings import USER_ID
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructField, StructType, StringType, FloatType, TimestampType, IntegerType
from cerebralcortex.data_importer.data_parsers import csv_data_parser
from cerebralcortex.data_importer import import_dir
from pyspark.sql.functions import minute, second, mean, window
from pyspark.sql import functions as F

# Import Data

In [ ]:
# import CSV/IoT data (demo stream name - demo-location-data-stream)
import_dir(
    cc_config="/home/md2k/cc_conf/",
    input_data_dir="sample_data/",
    user_id=USER_ID,
    data_file_extension=[".csv"],
    data_parser=csv_data_parser,
    gen_report=True
)

## Create CerebralCortex object

In [7]:
CC = Kernel("/home/md2k/cc_conf/")

## View Imported Data

In [15]:
location_stream = CC.get_stream("iot-data-stream")
location_stream.show(5, truncate=False)
#location_stream.plot()

+-------------------+-------------------+-------------------+-------+------------------------------------+
|timestamp          |localtime          |some_vals          |version|user                                |
+-------------------+-------------------+-------------------+-------+------------------------------------+
|2019-01-09 17:35:00|2019-01-09 17:35:00|0.0851887269487499 |1      |00000000-afb8-476e-9872-6472b4e66b68|
|2019-01-09 17:35:01|2019-01-09 17:35:01|0.16867549655743164|1      |00000000-afb8-476e-9872-6472b4e66b68|
|2019-01-09 17:35:02|2019-01-09 17:35:02|0.7404850816560419 |1      |00000000-afb8-476e-9872-6472b4e66b68|
|2019-01-09 17:35:03|2019-01-09 17:35:03|0.7131609970182962 |1      |00000000-afb8-476e-9872-6472b4e66b68|
|2019-01-09 17:35:04|2019-01-09 17:35:04|0.24253081438369195|1      |00000000-afb8-476e-9872-6472b4e66b68|
+-------------------+-------------------+-------------------+-------+------------------------------------+
only showing top 5 rows



In [45]:
df=location_stream.data

@pandas_udf("id long, v double", PandasUDFType.GROUPED_MAP)
def subtract_mean(pdf):
    # pdf is a pandas.DataFrame
    #v = pdf.v
    return pdf

#location_stream.compute_max().show(truncate=False)
#df.withColumn('d', fun_function(df.some_vals)).show()

df.groupBy("timestamp", window("timestamp", "2 seconds")).agg(df["mean"]).show()

# df2.groupBy("window").agg(F.mean('avg(some_vals)')).show(truncate=False)

AnalysisException: 'Cannot resolve column name "mean" among (timestamp, localtime, some_vals, version, user);'

In [5]:
location_stream.metadata

[{'name': 'iot-data-stream', 'description': 'this is a demo iot look a like stream', 'metadata_hash': '8eea2a10-96c6-3a17-a7a1-db87187b6b12', 'input_streams': [], 'annotations': [], 'data_descriptor': [<cerebralcortex.core.metadata_manager.stream.data_descriptor.DataDescriptor object at 0x7f7b834d1438>], 'modules': [<cerebralcortex.core.metadata_manager.stream.module_info.ModuleMetadata object at 0x7f7b834d14e0>], 'version': 1}]

## Write a smoothing algorithm

In [30]:
schema = StructType([
    StructField("ts", TimestampType())
    ])
@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def smooth_df(data):
#     df= data['some_vals'].ewm(span = 3600).mean()
#     print(type(df))
    df2 = pd.DataFrame([data.timestamp], columns=['ts'])
    return df2

## Run smoothing algorithm on imported data

In [31]:
#smooth_stream = location_stream.groupby("user").compute(smooth_df)
smooth_stream = location_stream.compute(smooth_df)
smooth_stream.show(3 truncate=False)

+----+
|  ts|
+----+
|null|
+----+



In [58]:
import pyspark.sql.functions
from pyspark.sql.functions import minute, second, mean, window

@pandas_udf('double', PandasUDFType.SCALAR)
def fun_function(a):
    return a+2
df=location_stream.data

#df.withColumn('d', fun_function(df.some_vals)).show()

df.groupBy("timestamp", window("timestamp", "5 minutes")).stddev("some_vals").show(5, False)

interval = 60
gdf = df.withColumn(
    'time_interval',
    pyspark.sql.functions.from_unixtime(pyspark.sql.functions.floor(pyspark.sql.functions.unix_timestamp(df['timestamp']) / interval) * interval)
).groupBy('time_interval')

#df.groupBy(second("timestamp").alias("hour")).agg(mean("some_vals").alias("mean")).show()

AttributeError: 'GroupedData' object has no attribute 'stddev'

## Visualize data

/opt/conda/lib/python3.6/site-packages/ipyleaflet/leaflet.py:80: DeprecationWarning:

Traits should be given as instances, not types (for example, `Int()`, not `Int`). Passing types is deprecated in traitlets 4.1.

/opt/conda/lib/python3.6/site-packages/ipyleaflet/leaflet.py:297: DeprecationWarning:

metadata {'dtype': None} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')

/opt/conda/lib/python3.6/site-packages/ipyleaflet/leaflet.py:416: DeprecationWarning:

metadata {'trait': <traitlets.traitlets.Instance object at 0x7f33837de320>} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')

/opt/conda/lib/python3.6/site-packages/ipyleaflet/leaflet.py:553: DeprecationWarning:

Traits should be given as instances, not types (for example, `Int()`, not `Int`). Passing types is deprecated in traitlets 4.1.

/opt

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [97]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.getOrCreate()

df = sparkSession.createDataFrame([(17.00, "2018-03-10T15:27:18+00:00"),
                            (13.00, "2018-03-11T12:27:18+00:00"),
                            (25.00, "2018-03-12T11:27:18.98+00:00"),
                            (20.00, "2018-03-13T15:27:18+00:00"),
                            (17.00, "2018-03-14T12:27:18+00:00"),
                            (99.00, "2018-03-15T11:27:17.56+00:00"),
                            (156.00, "2018-03-22T11:27:18+00:00"),
                            (17.00, "2018-03-31T11:27:18+00:00"),
                            (25.00, "2018-03-15T11:27:18+00:00"),
                            (25.00, "2018-03-16T11:27:18+00:00")
                            ],
                           ["id", "ts"])
df = df.withColumn('ts', df.ts.cast('timestamp'))
df.show(3)
schema = StructType([
    StructField("id", IntegerType()),
    StructField("ts", TimestampType())
])


@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def some_udf(df):
    # some computation
    df = pd.DataFrame(([df["id"], df["ts"]]), columns=['id', 'ts'])
    return df

extended = (df.withColumn("groupby_col", 
                          F.concat(F.col("ts").cast("date"), F.lit("-"), F.hour(F.col("ts")), F.lit("-"), F.minute(F.col("ts")), F.lit("-"), F.second(F.col("ts"))).cast("string")
                         ))
extended.show(truncate=False)
#df.groupby('id', F.window("ts", "15 days")).apply(some_udf).show()

+----+--------------------+
|  id|                  ts|
+----+--------------------+
|17.0| 2018-03-10 15:27:18|
|13.0| 2018-03-11 12:27:18|
|25.0|2018-03-12 11:27:...|
+----+--------------------+
only showing top 3 rows

+-----+----------------------+-------------------+
|id   |ts                    |groupby_col        |
+-----+----------------------+-------------------+
|17.0 |2018-03-10 15:27:18   |2018-03-10-15-27-18|
|13.0 |2018-03-11 12:27:18   |2018-03-11-12-27-18|
|25.0 |2018-03-12 11:27:18.98|2018-03-12-11-27-18|
|20.0 |2018-03-13 15:27:18   |2018-03-13-15-27-18|
|17.0 |2018-03-14 12:27:18   |2018-03-14-12-27-18|
|99.0 |2018-03-15 11:27:17.56|2018-03-15-11-27-17|
|156.0|2018-03-22 11:27:18   |2018-03-22-11-27-18|
|17.0 |2018-03-31 11:27:18   |2018-03-31-11-27-18|
|25.0 |2018-03-15 11:27:18   |2018-03-15-11-27-18|
|25.0 |2018-03-16 11:27:18   |2018-03-16-11-27-18|
+-----+----------------------+-------------------+

